In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/after-feature-engin/credit_card_balance_agg.csv
/kaggle/input/after-feature-engin/bureau_merge_no_drop.csv
/kaggle/input/after-feature-engin/pos_agg.csv
/kaggle/input/after-feature-engin/prev_app_agg.csv
/kaggle/input/after-feature-engin/install_pay_agg.csv
/kaggle/input/dseb-64-data-preparation-final-project/dseb63_columns_description.csv
/kaggle/input/dseb-64-data-preparation-final-project/dseb63_sample_submission.csv
/kaggle/input/dseb-64-data-preparation-final-project/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/dseb63_previous_application.csv
/kaggle/input/dseb-64-data-preparation-final-project/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/dseb63_bureau_balance.csv
/kaggle/input/dseb-64-data-preparation-final-project/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/dseb63_credit_card_balance.csv
/kaggle/input/dseb-64-data-preparation-final-project/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/dseb6

In [2]:
bureau_df=pd.read_csv('/kaggle/input/dseb-64-data-preparation-final-project/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/dseb63_bureau.csv')

In [3]:
bureau_df.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_CURR
0,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,254629
1,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,254629
2,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,254629
3,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,254629
4,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,254629


In [4]:
bureau_balance_df=pd.read_csv('/kaggle/input/dseb-64-data-preparation-final-project/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/dseb63_bureau_balance.csv')

In [5]:
def group(df_to_agg, prefix, aggregations, aggregate_by='SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by='SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by=aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on=aggregate_by)
def one_hot_encoder(df, categorical_columns=None, nan_as_category=True):
    """Create a new column for each categorical value in categorical columns using get dummies. """
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns = [
            col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns,
                        dummy_na=nan_as_category)
    categorical_columns = [c for c in df.columns if c not in original_columns]
    return df, categorical_columns


def label_encoder(df, categorical_columns=None):
    """Encode categorical values as integers (0,1,2,3...) with pandas.factorize. """
    if not categorical_columns:
        categorical_columns = [
            col for col in df.columns if df[col].dtype == 'object']
    for col in categorical_columns:
        df[col], uniques = pd.factorize(df[col])
    return df, categorical_columns

BUREAU_AGG = {
    'SK_ID_BUREAU': ['nunique'],

    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
    'AMT_ANNUITY': ['mean'],

    'BUREAU_CREDIT_FACT_DIFF': ['min', 'max', 'mean'],
    'BUREAU_CREDIT_ENDDATE_DIFF': ['min', 'max', 'mean'],
    'BUREAU_CREDIT_DEBT_RATIO': ['min', 'max', 'mean'],
    'BUREAU_IS_DPD': ['mean', 'sum'],
    'BUREAU_IS_DPD_OVER120': ['mean', 'sum'],

    'CNT_CREDIT_PROLONG': ['sum'],

    'CREDIT_ACTIVE_Active': ['mean'],
    'CREDIT_ACTIVE_Closed': ['mean'],
    'CREDIT_ACTIVE_Sold': ['mean'],
    'CREDIT_DAY_OVERDUE': ['max', 'mean', 'min'],
    'CREDIT_TYPE_Consumer credit': ['mean'],
    'CREDIT_TYPE_Credit card': ['mean'],
    'CREDIT_TYPE_Car loan': ['mean'],
    'CREDIT_TYPE_Mortgage': ['mean'],
    'CREDIT_TYPE_Microloan': ['mean'],

    'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['mean'],
    'DAYS_ENDDATE_FACT': ['min', 'max', 'mean'],
    'DEBT_CREDIT_DIFF': ['mean', 'sum'],

    'ENDDATE_DIF': ['min', 'max', 'mean'],

    'LL_AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'LL_DEBT_CREDIT_DIFF': ['mean'],
    'LL_STATUS_12345': ['mean'],

    'MONTHS_BALANCE_MIN': ['min'],
    'MONTHS_BALANCE_MAX': ['max'],
    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],

    'STATUS_0': ['mean'],
    'STATUS_1': ['mean'],
    'STATUS_12345': ['mean'],
    'STATUS_C': ['mean'],
    'STATUS_X': ['mean'],
}

BUREAU_ACTIVE_AGG = {
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM': ['max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean'],

    'CREDIT_TO_ANNUITY_RATIO': ['mean'],

    'DAYS_CREDIT': ['max', 'mean'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max'],
    'DAYS_CREDIT_UPDATE': ['min', 'mean'],

    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],

    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
}

BUREAU_CLOSED_AGG = {
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'sum'],

    'DAYS_CREDIT': ['max', 'var'],
    'DAYS_CREDIT_ENDDATE': ['max'],
    'DAYS_CREDIT_UPDATE': ['max'],

    'DCREDIT_DOVERDUE_DIFF': ['mean'],  # credit overdue diff
    'DCREDIT_DENDFACT_DIFF': ['mean'],  # credit end fact diff
    'DUPDATE_DENDATE_DIFF': ['mean'],  # update end date diff
    'ENDDATE_DIF': ['mean'],
    'STATUS_12345': ['mean'],

}

BUREAU_LOAN_TYPE_AGG = {
    'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
    'AMT_CREDIT_SUM': ['mean', 'max'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'max'],


    'DAYS_CREDIT': ['mean', 'max'],
    'DAYS_CREDIT_ENDDATE': ['max'],

    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'DEBT_CREDIT_LIMIT_DIFF': ['mean'],
    'DEBT_CREDIT_OVERDUE_DIFF': ['mean']
}

BUREAU_TIME_AGG = {
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],

    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'DEBT_CREDIT_LIMIT_DIFF': ['mean'],
    'DEBT_CREDIT_OVERDUE_DIFF': ['mean'],

    'STATUS_0': ['mean'],
    'STATUS_12345': ['mean'],
}

import pandas as pd
import gc  # Import garbage collector for memory management

def bureau(df,path_to_data):
    """
    Process bureau_clean.csv and return a pandas dataframe with aggregated features.
    """
    bureau = df.copy()

        # Credit duration and credit/account end date difference
    bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + \
        bureau['DAYS_CREDIT_ENDDATE']
    bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - \
        bureau['DAYS_ENDDATE_FACT']
    bureau['DCREDIT_DOVERDUE_DIFF'] = bureau['DAYS_CREDIT'] - \
        bureau['CREDIT_DAY_OVERDUE']  # days credit overdue diff
    bureau['DCREDIT_DENDFACT_DIFF'] = bureau['DAYS_CREDIT'] - \
        bureau['DAYS_ENDDATE_FACT']  # days credit endfact diff
    bureau['DUPDATE_DENDATE_DIFF'] = bureau['DAYS_CREDIT_UPDATE'] - \
        bureau['DAYS_CREDIT_ENDDATE']  # days update enddate diff

    # Credit to debt ratio and difference
    bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / \
        bureau['AMT_CREDIT_SUM_DEBT']
    bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - \
        bureau['AMT_CREDIT_SUM_DEBT']
    bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / \
        bureau['AMT_ANNUITY']
    bureau['BUREAU_CREDIT_FACT_DIFF'] = bureau['DAYS_CREDIT'] - \
        bureau['DAYS_ENDDATE_FACT']
    bureau['BUREAU_CREDIT_ENDDATE_DIFF'] = bureau['DAYS_CREDIT'] - \
        bureau['DAYS_CREDIT_ENDDATE']
    bureau['BUREAU_CREDIT_DEBT_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / \
        bureau['AMT_CREDIT_SUM']
    bureau['DEBT_CREDIT_LIMIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - \
        bureau['AMT_CREDIT_SUM_LIMIT']
    bureau['DEBT_CREDIT_OVERDUE_DIFF'] = bureau['AMT_CREDIT_SUM'] - \
        bureau['AMT_CREDIT_SUM_OVERDUE']

    # CREDIT_DAY_OVERDUE :
    bureau['BUREAU_IS_DPD'] = bureau['CREDIT_DAY_OVERDUE'].apply(
        lambda x: 1 if x > 0 else 0)
    bureau['BUREAU_IS_DPD_OVER120'] = bureau['CREDIT_DAY_OVERDUE'].apply(
        lambda x: 1 if x > 120 else 0)

    # One-hot encoder
    bureau, _ = one_hot_encoder(bureau, nan_as_category=False)

    # Join bureau balance features
    bureau = bureau.merge(bureau_balance(path_to_data),
                          how='left', on='SK_ID_BUREAU')

    # Flag months with late payments (days past due)
    bureau['STATUS_12345'] = 0
    for i in range(1, 6):
        bureau['STATUS_12345'] += bureau[f'STATUS_{i}']

    # Aggregate by number of months in balance and merge with bureau (loan length agg)
    features = ['AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM',
                'AMT_CREDIT_SUM_DEBT', 'DEBT_PERCENTAGE',
                'DEBT_CREDIT_DIFF', 'STATUS_0', 'STATUS_12345']
    agg_length = bureau.groupby('MONTHS_BALANCE_SIZE')[
        features].mean().reset_index()
    agg_length.rename(
        {feat: 'LL_' + feat for feat in features}, axis=1, inplace=True)
    bureau = bureau.merge(agg_length, how='left', on='MONTHS_BALANCE_SIZE')
    del agg_length

    # General loans aggregations
    bureau_agg = group(bureau, 'BUREAU_', BUREAU_AGG)

    # Active and closed loans aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    bureau_agg = group_and_merge(
        active, bureau_agg, 'BUREAU_ACTIVE_', BUREAU_ACTIVE_AGG)
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    bureau_agg = group_and_merge(
        closed, bureau_agg, 'BUREAU_CLOSED_', BUREAU_CLOSED_AGG)
    del active, closed

    # Aggregations for the main loan types
    for credit_type in ['Consumer credit', 'Credit card', 'Mortgage', 'Car loan', 'Microloan']:
        type_df = bureau[bureau['CREDIT_TYPE_' + credit_type] == 1]
        prefix = 'BUREAU_' + \
            credit_type.split(' ', maxsplit=1)[0].upper() + '_'
        bureau_agg = group_and_merge(
            type_df, bureau_agg, prefix, BUREAU_LOAN_TYPE_AGG)
        del type_df

    # Time based aggregations: last x months
    for time_frame in [6, 12]:
        prefix = f"BUREAU_LAST{time_frame}M_"
        time_frame_df = bureau[bureau['DAYS_CREDIT'] >= -30*time_frame]
        bureau_agg = group_and_merge(
            time_frame_df, bureau_agg, prefix, BUREAU_TIME_AGG)
        del time_frame_df

    # Last loan max overdue
    sort_bureau = bureau.sort_values(by=['DAYS_CREDIT'])
    del bureau
    gr = sort_bureau.groupby('SK_ID_CURR')[
        'AMT_CREDIT_MAX_OVERDUE'].last().reset_index()
    gr.rename(
        {'AMT_CREDIT_MAX_OVERDUE': 'BUREAU_LAST_LOAN_MAX_OVERDUE'}, inplace=True)
    bureau_agg = bureau_agg.merge(gr, on='SK_ID_CURR', how='left')
    del gr, sort_bureau

    # Ratios: total debt/total credit and active loans debt/ active loans credit
    bureau_agg['BUREAU_DEBT_OVER_CREDIT'] = \
        bureau_agg['BUREAU_AMT_CREDIT_SUM_DEBT_SUM'] / \
        bureau_agg['BUREAU_AMT_CREDIT_SUM_SUM']
    bureau_agg['BUREAU_ACTIVE_DEBT_OVER_CREDIT'] = \
        bureau_agg['BUREAU_ACTIVE_AMT_CREDIT_SUM_DEBT_SUM'] / \
        bureau_agg['BUREAU_ACTIVE_AMT_CREDIT_SUM_SUM']

    gc.collect()
    return bureau_agg

def bureau_balance(path_to_data):
    ''' Process dseb63_bureau_balance.csv and return a pandas dataframe. '''
    bb = pd.read_csv(os.path.join(path_to_data, 'dseb63_bureau_balance.csv'))
    # Credit duration and credit/account end date difference
    bb, cat_cols = one_hot_encoder(bb, nan_as_category=False)

    # Calculate rate for each category with decay
    bb_processed = bb.groupby('SK_ID_BUREAU')[cat_cols].mean().reset_index()

    # Min, Max, Count and mean duration of payments (months)
    agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}
    bb_processed = group_and_merge(bb, bb_processed, '', agg, 'SK_ID_BUREAU')

    del bb
    gc.collect()
    return bb_processed


In [6]:
path_to_data='/kaggle/input/dseb-64-data-preparation-final-project/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/'
bureau_ft_en=bureau(bureau_df,path_to_data)

In [7]:
bb =bureau_balance(path_to_data)

In [8]:
bb.shape

(817395, 13)

In [9]:
bureau_ft_en.shape

(263491, 203)

In [10]:
def check_nan(col):
    num_nan = col.isnull().sum()
    num_per = num_nan/len(col) * 100
    col_miss = pd.concat([num_nan, num_per], axis=1, keys=['number_of_NaN', 'percentage_of_NaN']).sort_values(by='percentage_of_NaN', ascending = False).round(1)
    col_miss = col_miss[col_miss['number_of_NaN'] != 0]
    print('Num fields: ', col.shape[1])
    print('Num missing fields: ', col_miss.shape[0])
    return col_miss
check_nan(bureau_ft_en)

Num fields:  203
Num missing fields:  173


,number_of_NaN,percentage_of_NaN
BUREAU_MICROLOAN_AMT_CREDIT_MAX_OVERDUE_MEAN,262620,99.7
BUREAU_MICROLOAN_AMT_CREDIT_MAX_OVERDUE_MAX,262620,99.7
BUREAU_MICROLOAN_DEBT_CREDIT_LIMIT_DIFF_MEAN,262620,99.7
BUREAU_MICROLOAN_DAYS_CREDIT_ENDDATE_MAX,261182,99.1
BUREAU_MORTGAGE_AMT_CREDIT_MAX_OVERDUE_MEAN,261048,99.1
...,...,...
BUREAU_DAYS_CREDIT_ENDDATE_MIN,2249,0.9
BUREAU_DAYS_CREDIT_ENDDATE_MAX,2249,0.9
BUREAU_DEBT_OVER_CREDIT,1019,0.4
BUREAU_AMT_CREDIT_SUM_MAX,1,0.0


In [12]:
bureau_ft_en.to_csv('bureau_merge.csv')